In [1]:
%pip install ipywidgets IPython pyaudio thread transformers torch pydub


Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()

record_button = widgets.Button(
    description="RECORD",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description = "STOP",
    disabled =False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    messages.put(True)

    with output:
        display("Starting.......")
        record = Thread(target=record_microphone)
        record.start()


        transcribe = Thread(target=Speech_recognition, args=(output,))
        transcribe.start()

def stop_recording(data):
    with output:

        messages.get()
        display("stopped.")
        

record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)

Button(button_style='success', description='RECORD', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='STOP', icon='stop', style=ButtonStyle())

Output()

In [3]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS  = 20
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()

    stream = p.open(format= AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   input_device_index=1,
                   frame_per_buffer=chunk)
    frames = []


    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)

        if len(frame) >= (FRAME_RATE * RECORD_SECONDS ) / chunk:
            recordings.put(frames.copy())
            frames = []

    stream.stop_stream()
    stream.close()
    p.terminate()




NameError: name 'pyaudio' is not defined

In [13]:
import subprocess
import json
import os
import sys
from vosk import Model,KaldiRecognizer
model = Model(model_name="vosk-model-en-in-0.4")
rec = KaldiRecognizer(model,FRAME_RATE)
rec.SetWords(True)

def Speech_recognition(output):
    while not messages.empty():
        frames = recordings.get()

        rec.AcceptWaveForm(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]

        cased = subprocess.check_output("Python recasepunc/recasepunc.py predict recasepunc/checkpoint", shell=True, text=True, input=text)

        output.append_stdout(cased)
        time.sleep(1)